In [1]:
import geopandas as gpd
from pyiem.database import get_sqlalchemy_conn

with get_sqlalchemy_conn("postgis") as conn:
    df = gpd.read_postgis(
        """
WITH obs as (
 SELECT segid, sum(lead - valid) from
 (SELECT segid, valid,  lead(valid) OVER (PARTITION by segid ORDER by valid ASC),
 cond_code from roads_log WHERE valid > '2025-11-01' and valid < '2026-02-26') as foo
 WHERE cond_code > 0 GROUP by segid
)
 SELECT obs.sum, base.simple_geom from obs JOIN
 roads_base base on (obs.segid = base.segid) 

""",
        conn,
        geom_col="simple_geom",
    )

print(len(df.index))

1014


In [2]:
import datetime

sts = datetime.datetime(2025, 11, 1)
ets = datetime.datetime(2026, 2, 26)
period = (ets - sts).total_seconds()

In [19]:
import numpy as np

import matplotlib.colors as mpcolors
from pyiem.plot import MapPlot, get_cmap

cmap = get_cmap("jet")
clevs = np.arange(0, 15, 2)
norm = mpcolors.BoundaryNorm(clevs, cmap.N)
df["color"] = [
    x for x in cmap(norm(df["sum"].dt.total_seconds() / period * 100.0))
]

mp = MapPlot(
    title="1 Nov 2025 - 26 Feb 2026 Percent Time with Non-Seasonable Driving Condition",
    subtitle="Based on Iowa DOT Road Condition Reports, till 9 PM 26 Feb 2026",
    axisbg="white",
    sector="spherical_mercator",
    west=-96.5,
    east=-91.,
    south=40.5,
    north=43.5,
    # figsize=(12.8,7.68),
)

df.to_crs(mp.panels[0].crs).plot(
    aspect=None,
    ax=mp.panels[0].ax,
    color=df["color"],
    lw=3,
    zorder=10,
)

mp.draw_colorbar(clevs, cmap, norm, extend="max", units="percent")
#mp.drawcities()

mp.postprocess(filename="260227.png")
mp.close()